In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test_ids = test["PassengerId"]

data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
def clean(df):
    df = df.drop(["Cabin", "Name", "PassengerId", "Ticket"], axis = 1)

    meanCols = ["Age", "Fare", "SibSp", "Parch"]
    modeCols = ["Pclass", "Embarked", "Sex", ]

    for col in meanCols:
        df[col] = df[col].fillna(df[col].mean())
    for col in modeCols:
        df[col] = df[col].fillna(df[col].mode()[0])

    return df


In [5]:
data = clean(data)
test = clean(test)

In [6]:
data.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [7]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from sklearn import preprocessing

In [9]:
le = preprocessing.LabelEncoder()

cols = ["Sex", "Embarked"]

for col in cols:
    data[col] = le.fit_transform(data[col])
    test[col] = le.transform(test[col])
    print(le.classes_)

data.head()

['female' 'male']
['C' 'Q' 'S']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = data.drop(["Survived"], axis = 1)
y = data["Survived"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)

In [12]:
predictions = clf.predict(X_val)

from sklearn.metrics import accuracy_score
acc_score = accuracy_score(y_val, predictions)
acc_score

0.8100558659217877

In [13]:
submission_pred = clf.predict(test)

In [14]:
df = pd.DataFrame({"PassengerId":test_ids.values,
                    "Survived":submission_pred
                   })

In [15]:
df.to_csv("submission.csv", index=False)